In [1]:
!pip install arxiv

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6104 sha256=95bdbbd37e6837007ce70589c882e9cf768090752c721048ac5033e61c6aa752
  Stored in directory: c:\users\utkar\appdata\local\pip\cache\wheels\03\f5\1a\23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [5]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [6]:
wiki.name

'wikipedia'

In [17]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))
retriever=vectordb.as_retriever()
retriever

d:\Project\learning\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Project\learning\venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\utkar\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F5A79CD2B0>, search_kwargs={})

In [18]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [19]:

retriever_tool.name

'langsmith_search'

In [20]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [21]:
tools=[wiki,arxiv,retriever_tool]

In [22]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Project\\learning\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001F5853C1F80>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.v

In [32]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="gemma2:2b")

In [33]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

d:\Project\learning\venv\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [34]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [35]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [36]:
agent_executor.invoke({"input":"Tell me about U-net architecture"})



> Entering new AgentExecutor chain...
## The U-Net Architecture: A Powerful Tool for Image Segmentation

The U-Net architecture is a convolutional neural network (CNN) specifically designed for **image segmentation**. This means it can identify and label individual objects within an image. 

**Imagine this:** You have a photo of a bustling city, and you want to precisely extract the boundaries of buildings, roads, parks, and other features. That's where U-Net comes in! 

**Key Features:**

1. **"U" Shape:** The architecture takes its name from its distinctive "U" shape:
    * ** contracting path (encoder):**  Starts with multiple convolutional layers stacked together, gradually reducing the image resolution while extracting features. Think of it as a series of filters looking for specific details. 
    * ** expanding path (decoder):**  Converts the downsampled feature maps back into higher-resolution images by upsampling them and adding connections to generate a complete segmentation

{'input': 'Tell me about U-net architecture',
 'output': '## The U-Net Architecture: A Powerful Tool for Image Segmentation\n\nThe U-Net architecture is a convolutional neural network (CNN) specifically designed for **image segmentation**. This means it can identify and label individual objects within an image. \n\n**Imagine this:** You have a photo of a bustling city, and you want to precisely extract the boundaries of buildings, roads, parks, and other features. That\'s where U-Net comes in! \n\n**Key Features:**\n\n1. **"U" Shape:** The architecture takes its name from its distinctive "U" shape:\n    * ** contracting path (encoder):**  Starts with multiple convolutional layers stacked together, gradually reducing the image resolution while extracting features. Think of it as a series of filters looking for specific details. \n    * ** expanding path (decoder):**  Converts the downsampled feature maps back into higher-resolution images by upsampling them and adding connections to gen

In [37]:
agent_executor.invoke({"input":"Teach langchain to a 5 year old"})



> Entering new AgentExecutor chain...
Imagine you have a super smart toy robot friend!  

This robot can understand and answer questions just like you do, but it learned all about the world from reading lots of books and listening to grown-ups. 🤖📚

LangChain is like giving that robot special tools so it can remember things better and find even more answers! 🧰  

* **Ask it a question:** "What's the color of the sky?" Langchain helps it find information about the sky in its memory, then give you an answer!
* **Make it tell stories:** "Tell me a story about a dinosaur!" Langchain finds all those exciting facts and stories from books and makes one-up for you.  🦖📚 

LangChain lets our robots friend learn, remember, and answer questions like us – and all that learning is really fun! 🎉


**So basically, LangChain helps our robot friends understand the world better and do cool things with it! 🤩** 


> Finished chain.


{'input': 'Teach langchain to a 5 year old',
 'output': 'Imagine you have a super smart toy robot friend!  \n\nThis robot can understand and answer questions just like you do, but it learned all about the world from reading lots of books and listening to grown-ups. 🤖📚\n\nLangChain is like giving that robot special tools so it can remember things better and find even more answers! 🧰  \n\n* **Ask it a question:** "What\'s the color of the sky?" Langchain helps it find information about the sky in its memory, then give you an answer!\n* **Make it tell stories:** "Tell me a story about a dinosaur!" Langchain finds all those exciting facts and stories from books and makes one-up for you.  🦖📚 \n\nLangChain lets our robots friend learn, remember, and answer questions like us – and all that learning is really fun! 🎉\n\n\n**So basically, LangChain helps our robot friends understand the world better and do cool things with it! 🤩** \n'}